In [1]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='JSONs\\szakdoga-487511-77e98d2837e4.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/11GIgSrcKP407AeyQA6SLQYg3d52GyVEl-QrXojluKwI/edit?usp=sharing')

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests
import pandas as pd


api_key = os.getenv('riot_api_key')
puuid = 'd_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A'

def get_puuid(gameName, tagLine, api_key):
    link = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid, api_key):
    link = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'
    response = requests.get(link)
    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']
    return f'{gameName}#{tagLine}'

def get_match_ids(puuid, start, count, api_key):
    link = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start={start}&count={count}&api_key={api_key}'
    response = requests.get(link)
    return response.json()

def get_match_info(match_id, api_key):
    link = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    return response.json()

def process_match_data(gameData, puuid):
    #ARCHITECTURE
    metadata = gameData['metadata']
    info = gameData['info']
    players = info['participants']

    match_id = metadata['matchId']
    participants = info['participants']
    teams = info['teams']

    player = None
    for p in players:
        if p.get('puuid') == puuid:
            player = p
            break
    
    if player is None:
        return None

    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']
    primary = styles[0]
    secondary = styles[1]

    side_dict = {100: 'blue', 200: 'red'}
    side = side_dict[player['teamId']]

    #GAME INFO
    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    gameStartTimestamp = info['gameStartTimestamp']
    patch = info['gameVersion']
    gameEndTimestamp = info['gameEndTimestamp']



    #SURRENDER
    eralySurrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    #PLAYER INFO
    riotID = player['riotIdGameName']
    RiotTagLine = player['riotIdTagline']
    summonerID = player['summonerId']
    summonerName = player['summonerName']

    #CHAMPION INFO
    role = player['individualPosition']
    champlevel = player['champLevel']
    championName = player['championName']

    assists = player['assists']
    kills = player['kills']
    deaths = player['deaths']
    firstBloodKill = player['firstBloodKill']

    goldEarned = player['goldEarned']
    cs = player['totalMinionsKilled'] + player['neutralMinionsKilled']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner1Id = player['summoner1Id']
    summoner2Id = player['summoner2Id']

    #PLAYER PERFORMANCE
    totalDmg = player['totalDamageDealtToChampions']
    pyiscalDmg = player['physicalDamageDealtToChampions']
    magicDmg = player['magicDamageDealtToChampions']
    trueDmg = player['trueDamageDealtToChampions']
    cc = player['totalTimeCCDealt']

    #VISION
    visionScore = player['visionScore']

    win = player['win']



    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']


    primary_style = primary['style']
    secondary_style = secondary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']
    #secondary_perk_3 = secondary['selections'][2]['perk']

    for team in teams:
        bans = team['bans']

        if team['teamId'] == player['teamId']:
            objs = team['objectives']
            barons = objs['baron']
            dragons = objs['dragon']
            grubs = objs['horde']
            herald = objs['riftHerald']
            inhibitor = objs['inhibitor']
            tower = objs['tower']


    matchDf = pd.DataFrame({
        'matchId': [match_id],
        'gameCreation': [game_creation],
        'gameDuration': [game_duration],
        'gameStartTimestamp': [gameStartTimestamp],
        'gameEndTimestamp': [gameEndTimestamp],
        'patch': [patch],
        'puuid': [puuid],
        'eralySurrender': [eralySurrender],
        'surrender': [surrender],
        'riotID': [riotID],
        'RiotTagLine': [RiotTagLine],
        'summonerID': [summonerID],
        'summonerName': [summonerName],
        'side': [side],
        'win': [win],
        'championName': [championName],
        'champLevel': [champlevel],
        'role': [role],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'firstBloodKill': [firstBloodKill],
        'goldEarned': [goldEarned],
        'cs': [cs],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'summoner1Id': [summoner1Id],
        'summoner2Id': [summoner2Id],
        'totalDmg': [totalDmg],
        'pyiscalDmg': [pyiscalDmg],
        'magicDmg': [magicDmg],
        'trueDmg': [trueDmg],
        'cc': [cc],
        'visionScore': [visionScore],
        'defense': [defense],
        'flex': [flex],
        'offense': [offense],
        'primary_style': [primary_style],
        'secondary_style': [secondary_style],
        'primary_keystone': [primary_keystone],
        'primary_perk_1': [primary_perk_1],
        'primary_perk_2': [primary_perk_2],
        'primary_perk_3': [primary_perk_3],
        'secondary_perk_1': [secondary_perk_1],
        'secondary_perk_2': [secondary_perk_2],

    })
    
    return matchDf

def replace_ids_with_names(final_df):
    import requests
    import pandas as pd

    url_perks = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
    url_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
    url_items = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

    perks_data = requests.get(url_perks).json()
    styles_data = requests.get(url_styles).json()
    items_data = requests.get(url_items).json()

    perk_dict = {item['id']: item['name'] for item in perks_data if 'id' in item and 'name' in item}

    styles_list = styles_data.get('styles', []) if isinstance(styles_data, dict) else styles_data
    style_dict = {item['id']: item['name'] for item in styles_list if 'id' in item and 'name' in item}

    item_dict = {item['id']: item['name'] for item in items_data if 'id' in item and 'name' in item}

    perk_cols = [
        'primary_keystone', 'primary_perk_1', 'primary_perk_2', 'primary_perk_3',
        'secondary_perk_1', 'secondary_perk_2'
    ]
    style_cols = ['primary_style', 'secondary_style']

    item_cols = [f'item{i}' for i in range(7)]

    for col in perk_cols:
        if col in final_df.columns:
            final_df[col] = final_df[col].map(perk_dict).fillna(final_df[col])

    for col in style_cols:
        if col in final_df.columns:
            final_df[col] = final_df[col].map(style_dict).fillna(final_df[col])

    for col in item_cols:
        if col in final_df.columns:
            final_df[col] = final_df[col].map(item_dict).fillna(final_df[col])

    return final_df

In [4]:
import pandas as pd
from requests_futures.sessions import FuturesSession
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import requests
import time

# --- BEÁLLÍTÁSOK ---
api_key = os.getenv('riot_api_key')
puuid = "d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A"
region = "asia"

# --- 1. GOLYÓÁLLÓ RETRY STRATÉGIA ---
retry_strategy = Retry(
    total=5, 
    status_forcelist=[429, 500, 502, 503, 504], 
    backoff_factor=1, 
    respect_retry_after_header=True
)
adapter = HTTPAdapter(max_retries=retry_strategy)

sync_session = requests.Session()
sync_session.mount("https://", adapter)

async_session = FuturesSession(max_workers=10)
async_session.mount("https://", adapter)

headers = {"X-Riot-Token": api_key}
df_lista = []

start = 0
count = 100
found_patch_16 = False

print("Meccsek letöltése a 16-os szezonból...")

# --- 2. OKOS LAPOZÁS (PAGINATION) ---
should_break = False
while True:
    print(f"ID-k lekérése: {start} - {start + count}...")
    
    ids_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": start, 
        "count": count, 
        "queue": 420, 
    } 
    
    response = sync_session.get(ids_url, headers=headers, params=params)
    response.raise_for_status()
    match_ids = response.json()
    
    if not match_ids:
        print("Minden meccs letöltve ehhez a szezonhoz!")
        break
        
    # --- 3. MECCSEK LETÖLTÉSE EGYSZERRE (KONKURENCIA) ---
    futures_dict = {}
    for match_id in match_ids:
        match_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
        futures_dict[match_id] = async_session.get(match_url, headers=headers)
        
    # --- 4. MECCSEK FELDOLGOZÁSA ---
    for match_id in match_ids:
        try:
            resp = futures_dict[match_id].result()
            resp.raise_for_status()
            game = resp.json()
            
            if 'metadata' not in game:
                continue
                
            patch_version = game['info']['gameVersion']
            major_patch = int(patch_version.split('.')[0])
            
            if major_patch < 16:
                print(f"Leálltam, mert találtam már 16-os patch-et, de most {patch_version} jött")
                should_break = True
                break
                
            matchDf = process_match_data(game, puuid)
            if matchDf is not None:
                df_lista.append(matchDf)
                
        except Exception as e:
            print(f"Hiba a {match_id} feldolgozásakor: {e}")

    start += count
    
    if should_break:
        break

# --- 5. VÉGEREDMÉNY ÖSSZEFŰZÉSE ---
if df_lista:
    final_df = pd.concat(df_lista, ignore_index=True)
    print(f"\nSiker! Összesen {len(final_df)} db meccs bekerült az adatbázisba.")
else:
    final_df = pd.DataFrame()
    print("\nEgyetlen meccset sem találtunk a 16-os patch-ből.")

Meccsek letöltése a 16-os szezonból...
ID-k lekérése: 0 - 100...
ID-k lekérése: 100 - 200...
Leálltam, mert találtam már 16-os patch-et, de most 15.24.734.7485 jött

Siker! Összesen 157 db meccs bekerült az adatbázisba.


In [8]:
final_df = replace_ids_with_names(final_df)

In [9]:
final_df

,matchId,gameCreation,gameDuration,gameStartTimestamp,gameEndTimestamp,patch,puuid,eralySurrender,surrender,riotID,...,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,KR_8098225521,1771590376385,1449,1771590391837,1771591841050,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5005,Domination,Sorcery,Electrocute,Taste of Blood,Grisly Mementos,Ultimate Hunter,Manaflow Band,Scorch
1,KR_8097162372,1771521070081,1345,1771521089390,1771522434394,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5008,Sorcery,Resolve,Phase Rush,Manaflow Band,Transcendence,Scorch,Second Wind,Overgrowth
2,KR_8097124375,1771519379526,1376,1771519390673,1771520767094,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
3,KR_8097071976,1771517433020,1442,1771517457397,1771518899182,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
4,KR_8097025934,1771515805194,1371,1771515821712,1771517193015,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,KR_8016411694,1767891836909,1856,1767891857792,1767893714208,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5008,Domination,Resolve,Electrocute,Sudden Impact,Sixth Sense,Ultimate Hunter,Bone Plating,Overgrowth
153,KR_8016368997,1767890288378,1137,1767890307812,1767891445209,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Domination,Sorcery,Electrocute,Taste of Blood,Sixth Sense,Ultimate Hunter,Manaflow Band,Scorch
154,KR_8016277368,1767886809010,2651,1767886826915,1767889477748,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5010,5007,Resolve,Inspiration,Aftershock,Demolish,Second Wind,Overgrowth,Hextech Flashtraption,Cosmic Insight
155,KR_8016195389,1767884657565,1827,1767884682810,1767886510070,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5008,Domination,Resolve,Electrocute,Sudden Impact,Grisly Mementos,Ultimate Hunter,Second Wind,Overgrowth


In [13]:
from pangres import upsert
from sqlalchemy import text, create_engine

db_username = os.getenv('db_username')
db_host = os.getenv('db_host')
db_port = os.getenv('db_port')
db_name = os.getenv('db_name')
db_password = os.getenv('db_password')

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
    conn_url = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
    return conn_url

conn_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine =  create_engine(conn_url, pool_recycle=3600)

connection = db_engine.connect()


In [14]:
final_df['uuid'] = final_df['matchId'] + '_' + final_df['puuid']
final_df = final_df.set_index('uuid')

In [15]:
final_df

,matchId,gameCreation,gameDuration,gameStartTimestamp,gameEndTimestamp,patch,puuid,eralySurrender,surrender,riotID,...,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
uuid,,,,,,,,,,,,,,,,,,,,,
KR_8098225521_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8098225521,1771590376385,1449,1771590391837,1771591841050,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5005,Domination,Sorcery,Electrocute,Taste of Blood,Grisly Mementos,Ultimate Hunter,Manaflow Band,Scorch
KR_8097162372_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8097162372,1771521070081,1345,1771521089390,1771522434394,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5008,Sorcery,Resolve,Phase Rush,Manaflow Band,Transcendence,Scorch,Second Wind,Overgrowth
KR_8097124375_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8097124375,1771519379526,1376,1771519390673,1771520767094,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
KR_8097071976_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8097071976,1771517433020,1442,1771517457397,1771518899182,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
KR_8097025934_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8097025934,1771515805194,1371,1771515821712,1771517193015,16.4.746.5697,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Precision,Resolve,Lethal Tempo,Triumph,Legend: Alacrity,Cut Down,Bone Plating,Unflinching
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KR_8016411694_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8016411694,1767891836909,1856,1767891857792,1767893714208,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5008,5008,Domination,Resolve,Electrocute,Sudden Impact,Sixth Sense,Ultimate Hunter,Bone Plating,Overgrowth
KR_8016368997_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8016368997,1767890288378,1137,1767890307812,1767891445209,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,True,Hide on bush,...,5008,5005,Domination,Sorcery,Electrocute,Taste of Blood,Sixth Sense,Ultimate Hunter,Manaflow Band,Scorch
KR_8016277368_d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDWdMWCdZHP57iSVlOxKyHqgo903spXcj8A,KR_8016277368,1767886809010,2651,1767886826915,1767889477748,16.1.734.5787,d_XLGKapqHkqDAyH9AT5fRaC2mqJq0MZ9o12WwHaXr-xDW...,False,False,Hide on bush,...,5010,5007,Resolve,Inspiration,Aftershock,Demolish,Second Wind,Overgrowth,Hextech Flashtraption,Cosmic Insight


In [16]:
upsert(con=connection, df=final_df,schema='soloq', table_name='player_matches', create_table=True,create_schema=True, if_row_exists='update')


In [17]:
connection.commit()

In [18]:
def get_player_matches(puuid):
    with db_engine.connect() as connection:
        df = pd.read_sql(text(f"""SELECT * 
                                FROM soloq.player_matches
                                where puuid = '{puuid}'"""), con=connection)
    return df